In [1]:
!pip install streamlit sqlalchemy smolagents python-dotenv smolagents[litellm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 33.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1


In [2]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 5s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [3]:
%%writefile ai_text_to_sql_agent.py

import streamlit as st
import pandas as pd
import os
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, Float, inspect, text, insert
from smolagents import tool, CodeAgent, LiteLLMModel
from dotenv import load_dotenv

# --- UI SETUP ---
st.set_page_config(page_title="AI Text-to-SQL Agent", layout="wide")
st.title("🧑‍💻 AI Text-to-SQL Agent")
st.write("Upload your CSV or try a demo. Ask questions in natural language and get SQL + results!")

# --- OPENAI API KEY INPUT ---
with st.sidebar:
    st.header("🔑 OpenAI API Key")
    openai_api_key = st.text_input("Enter your OpenAI API Key", type="password")
    st.markdown("[Get an OpenAI API key](https://platform.openai.com/api-keys)")

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    load_dotenv()

# --- DEMO DATASETS ---
demo_datasets = {
    "Receipts": pd.DataFrame([
        {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
        {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
        {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
        {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
    ]),
    "Waiters": pd.DataFrame([
        {"receipt_id": 1, "waiter_name": "Corey Johnson"},
        {"receipt_id": 2, "waiter_name": "Michael Watts"},
        {"receipt_id": 3, "waiter_name": "Michael Watts"},
        {"receipt_id": 4, "waiter_name": "Margaret James"},
    ]),
}

# --- DATA UPLOAD/SELECTION ---
st.subheader("1. Select or Upload Data")
dataset_option = st.radio("Choose a dataset:", ["Upload CSV"] + list(demo_datasets.keys()))

uploaded_file = None
user_df = None
if dataset_option == "Upload CSV":
    uploaded_file = st.file_uploader("Upload your CSV file", type=["csv"])
    if uploaded_file:
        user_df = pd.read_csv(uploaded_file)
        st.write("Preview of uploaded data:")
        st.dataframe(user_df.head())
else:
    user_df = demo_datasets[dataset_option]
    st.write(f"Preview of demo data: {dataset_option}")
    st.dataframe(user_df.head())

# --- SQLALCHEMY SETUP ---
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()
tables = {}

def create_table_from_df(df, table_name):
    columns = []
    for col, dtype in zip(df.columns, df.dtypes):
        if pd.api.types.is_integer_dtype(dtype):
            columns.append(Column(col, Integer))
        elif pd.api.types.is_float_dtype(dtype):
            columns.append(Column(col, Float))
        else:
            columns.append(Column(col, String(128)))
    table = Table(table_name, metadata_obj, *columns)
    metadata_obj.create_all(engine)
    # Insert rows
    for _, row in df.iterrows():
        stmt = insert(table).values(**row.to_dict())
        with engine.begin() as connection:
            connection.execute(stmt)
    return table

# Clear metadata for new uploads
metadata_obj.clear()

if user_df is not None:
    # If demo, may need to add both tables
    if dataset_option == "Receipts":
        create_table_from_df(demo_datasets["Receipts"], "receipts")
        create_table_from_df(demo_datasets["Waiters"], "waiters")
    else:
        create_table_from_df(user_df, "user_table")

# --- TOOL DEFINITION ---
def get_table_descriptions():
    inspector = inspect(engine)
    desc = ""
    for table_name in inspector.get_table_names():
        columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table_name)]
        table_description = f"Table '{table_name}':\n"
        table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
        desc += "\n\n" + table_description
    return desc

@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the available tables. Beware that this tool's output is a string representation of the execution output.

    Args:
        query: The SQL query to execute on the database.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

sql_engine.description = (
    "Allows you to perform SQL queries on the available tables. Beware that this tool's output is a string representation of the execution output."
    + get_table_descriptions() +
    "\n\nArgs:\n    query: The query to perform. This should be correct SQL.\n"
)

# --- AGENT SETUP ---
def get_agent():
    model = LiteLLMModel(model_id="openai/gpt-4o")
    return CodeAgent(tools=[sql_engine], model=model)

# --- MAIN APP LOGIC ---
st.subheader("2. Ask a Question")
question = st.text_input("Enter your question (e.g. Who got the most expensive receipt?)")

if st.button("Run Query") and question and user_df is not None and openai_api_key:
    with st.spinner("Generating SQL and running query..."):
        agent = get_agent()
        try:
            result = agent.run(question)
            st.success("Query executed!")
            st.write("**Agent Output:**")
            st.code(result, language="text")
        except Exception as e:
            st.error(f"Error: {e}")
elif user_df is None:
    st.info("Please upload a CSV or select a demo dataset.")
elif not openai_api_key:
    st.info("Please enter your OpenAI API key in the sidebar.")

Writing ai_text_to_sql_agent.py


In [4]:
!streamlit run /content/ai_text_to_sql_agent.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.243.193.16
⠙your url is: https://salty-animals-roll.loca.lt
